# Option B: Multimodal Document Intelligence

**Module:** 4.3 - Capstone Project (Domain 4: Production AI)
**Time:** 35-45 hours total
**Difficulty:** ⭐⭐⭐⭐⭐

---

## 🎯 Project Overview

Build a system for understanding complex documents with:
- **Document Ingestion** - PDFs, images, scanned documents
- **Vision-Language Model** - Understand diagrams, charts, layouts
- **Structured Extraction** - Pull data into structured formats
- **Question Answering** - Answer questions about documents
- **Export Pipeline** - Output to JSON, CSV, or databases

---

## 🎯 Learning Objectives

By completing this project, you will:
- [ ] Process PDFs and images for AI analysis
- [ ] Use Vision-Language Models for document understanding
- [ ] Extract structured information from unstructured documents
- [ ] Build a multimodal RAG system
- [ ] Create evaluation benchmarks for document AI

---

## 🌍 Real-World Context

Document intelligence is transforming industries:

| Industry | Use Case | Impact |
|----------|----------|--------|
| **Finance** | Invoice processing, KYC | 80% faster processing |
| **Healthcare** | Medical record extraction | Better patient care |
| **Legal** | Contract analysis | Hours→Minutes for review |
| **Research** | Paper analysis, data extraction | Accelerated discovery |

Companies like AWS (Textract), Google (Document AI), and Microsoft (Form Recognizer) have built billion-dollar products around document intelligence.

---

## 🧒 ELI5: What is Document Intelligence?

> **Imagine you're a librarian** who needs to organize a million books.
>
> A regular computer can only read the typed words. But what about:
> - **Handwritten notes** in the margins?
> - **Diagrams and charts** explaining concepts?
> - **Tables** with important data?
> - **Photos** included in the text?
>
> **Document Intelligence** is like giving a computer eyes AND understanding. It can:
> 1. **See** the document like a photo
> 2. **Read** all the text (printed or handwritten)
> 3. **Understand** charts, diagrams, and layouts
> 4. **Extract** the important information
> 5. **Answer questions** about what it found
>
> It's the difference between a scanner (just copies) and a smart assistant (understands and helps).

---

## 🏗️ System Architecture

```
┌─────────────────────────────────────────────────────────────────────┐
│                  Multimodal Document Intelligence                    │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  ┌──────────────┐    ┌──────────────┐    ┌──────────────┐          │
│  │   Document   │───▶│   Ingestion  │───▶│   Analysis   │          │
│  │   Upload     │    │   Pipeline   │    │   Engine     │          │
│  └──────────────┘    └───────┬──────┘    └───────┬──────┘          │
│                              │                   │                  │
│         ┌────────────────────┼───────────────────┘                  │
│         │                    │                                      │
│         ▼                    ▼                                      │
│  ┌──────────────┐    ┌──────────────┐    ┌──────────────┐          │
│  │    OCR /     │    │    Vision    │    │  Structured  │          │
│  │  Text Ext.   │    │   Language   │    │  Extraction  │          │
│  └──────────────┘    │    Model     │    └───────┬──────┘          │
│                      └──────────────┘            │                  │
│                                                  ▼                  │
│  ┌──────────────┐    ┌──────────────┐    ┌──────────────┐          │
│  │  Multimodal  │◀───│     QA       │◀───│   Export     │          │
│  │     RAG      │    │   Engine     │    │   (JSON/CSV) │          │
│  └──────────────┘    └──────────────┘    └──────────────┘          │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

---

## Part 1: Environment & Model Selection

In [ ]:
# Environment Setup
import torch
from datetime import datetime

print("🚀 OPTION B: MULTIMODAL DOCUMENT INTELLIGENCE")
print("="*60)
print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print(f"\nGPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'Not available'}")
print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Vision-Language Model Options
VLM_OPTIONS = {
    "llava-1.6-34b": {
        "name": "LLaVA 1.6 34B",
        "memory_bf16": 68,
        "memory_int4": 17,
        "strengths": ["General document understanding", "OCR capability", "Chart analysis"],
    },
    "qwen2-vl-72b": {
        "name": "Qwen2-VL 72B",
        "memory_bf16": 144,
        "memory_int4": 36,
        "strengths": ["Multilingual", "High resolution", "Document layout"],
    },
    "internvl2-26b": {
        "name": "InternVL2 26B",
        "memory_bf16": 52,
        "memory_int4": 13,
        "strengths": ["Fast inference", "Good for tables", "Multi-image"],
    },
}

print("\n📦 Vision-Language Model Options:")
for key, model in VLM_OPTIONS.items():
    print(f"\n  {model['name']}")
    print(f"    Memory (INT4): {model['memory_int4']} GB")
    print(f"    Strengths: {', '.join(model['strengths'])}")

In [ ]:
# Project Configuration

PROJECT_NAME = "doc-intelligence"
VLM_MODEL = "llava-hf/llava-v1.6-34b-hf"  # Good balance of quality and speed
EMBEDDING_MODEL = "BAAI/bge-m3"

# Document types to support
SUPPORTED_FORMATS = [".pdf", ".png", ".jpg", ".jpeg", ".tiff", ".bmp"]

# Extraction schemas (define what to extract)
EXTRACTION_SCHEMAS = {
    "invoice": {
        "vendor_name": "string",
        "invoice_number": "string",
        "date": "date",
        "total_amount": "number",
        "line_items": [{"description": "string", "quantity": "number", "price": "number"}],
    },
    "research_paper": {
        "title": "string",
        "authors": ["string"],
        "abstract": "string",
        "key_findings": ["string"],
        "methodology": "string",
    },
    "generic": {
        "title": "string",
        "content_summary": "string",
        "key_entities": ["string"],
        "tables": [{"description": "string", "data": "object"}],
    },
}

print(f"✅ Configuration set")
print(f"   VLM: {VLM_MODEL}")
print(f"   Formats: {', '.join(SUPPORTED_FORMATS)}")
print(f"   Schemas: {', '.join(EXTRACTION_SCHEMAS.keys())}")

---

## Part 2: Document Ingestion Pipeline

In [ ]:
# Document Ingestion System

from pathlib import Path
from typing import List, Dict, Any, Optional, Union
from dataclasses import dataclass
from PIL import Image
import io

@dataclass
class DocumentPage:
    """A single page from a document."""
    page_number: int
    image: Image.Image
    text: Optional[str] = None
    width: int = 0
    height: int = 0
    
@dataclass
class Document:
    """A complete document with all pages."""
    path: str
    pages: List[DocumentPage]
    metadata: Dict[str, Any]
    doc_type: str = "generic"

class DocumentIngester:
    """
    Ingests documents from various formats into a unified structure.
    """
    
    def __init__(self, dpi: int = 150):
        self.dpi = dpi
        self._pdf_lib = None
    
    def ingest(self, path: Union[str, Path]) -> Document:
        """
        Ingest a document from file path.
        
        Args:
            path: Path to document file
            
        Returns:
            Document object with all pages
        """
        path = Path(path)
        
        if not path.exists():
            raise FileNotFoundError(f"Document not found: {path}")
        
        suffix = path.suffix.lower()
        
        if suffix == ".pdf":
            return self._ingest_pdf(path)
        elif suffix in [".png", ".jpg", ".jpeg", ".tiff", ".bmp"]:
            return self._ingest_image(path)
        else:
            raise ValueError(f"Unsupported format: {suffix}")
    
    def _ingest_pdf(self, path: Path) -> Document:
        """Ingest a PDF document."""
        try:
            import fitz  # PyMuPDF
        except ImportError:
            print("⚠️ PyMuPDF not installed. Install with: pip install pymupdf")
            raise
        
        doc = fitz.open(str(path))
        pages = []
        
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            
            # Render to image
            mat = fitz.Matrix(self.dpi / 72, self.dpi / 72)
            pix = page.get_pixmap(matrix=mat)
            img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            
            # Extract text
            text = page.get_text()
            
            pages.append(DocumentPage(
                page_number=page_num + 1,
                image=img,
                text=text,
                width=pix.width,
                height=pix.height,
            ))
        
        doc.close()
        
        return Document(
            path=str(path),
            pages=pages,
            metadata={
                "filename": path.name,
                "num_pages": len(pages),
                "format": "pdf",
            }
        )
    
    def _ingest_image(self, path: Path) -> Document:
        """Ingest an image document."""
        img = Image.open(path)
        
        # Convert to RGB if necessary
        if img.mode != "RGB":
            img = img.convert("RGB")
        
        page = DocumentPage(
            page_number=1,
            image=img,
            width=img.width,
            height=img.height,
        )
        
        return Document(
            path=str(path),
            pages=[page],
            metadata={
                "filename": path.name,
                "num_pages": 1,
                "format": path.suffix[1:],
                "size": (img.width, img.height),
            }
        )

print("✅ Document ingestion system ready")
print("\nUsage:")
print("  ingester = DocumentIngester()")
print("  doc = ingester.ingest('path/to/document.pdf')")

---

## Part 3: Vision-Language Model Integration

In [ ]:
# Vision-Language Model Wrapper

class VisionLanguageModel:
    """
    Wrapper for Vision-Language Models.
    
    Supports multiple VLM backends with a unified interface.
    """
    
    def __init__(
        self,
        model_name: str = "llava-hf/llava-v1.6-34b-hf",
        load_in_4bit: bool = True,
    ):
        self.model_name = model_name
        self.load_in_4bit = load_in_4bit
        self._model = None
        self._processor = None
    
    def _load(self):
        """Lazy load the model."""
        if self._model is not None:
            return
        
        from transformers import (
            AutoModelForVision2Seq,
            AutoProcessor,
            BitsAndBytesConfig,
        )
        
        print(f"Loading VLM: {self.model_name}")
        
        if self.load_in_4bit:
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.bfloat16,
                bnb_4bit_quant_type="nf4",
            )
            self._model = AutoModelForVision2Seq.from_pretrained(
                self.model_name,
                quantization_config=bnb_config,
                device_map="auto",
                trust_remote_code=True,
            )
        else:
            self._model = AutoModelForVision2Seq.from_pretrained(
                self.model_name,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                trust_remote_code=True,
            )
        
        self._processor = AutoProcessor.from_pretrained(self.model_name)
        print(f"✅ VLM loaded")
    
    def analyze_image(
        self,
        image: Image.Image,
        prompt: str,
        max_new_tokens: int = 1024,
    ) -> str:
        """
        Analyze an image with a given prompt.
        
        Args:
            image: PIL Image to analyze
            prompt: Question or instruction
            max_new_tokens: Maximum response length
            
        Returns:
            Model's response
        """
        self._load()
        
        # Format prompt with image token
        full_prompt = f"<image>\nUSER: {prompt}\nASSISTANT:"
        
        inputs = self._processor(
            text=full_prompt,
            images=image,
            return_tensors="pt"
        ).to("cuda")
        
        with torch.no_grad():
            outputs = self._model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=False,
            )
        
        response = self._processor.decode(outputs[0], skip_special_tokens=True)
        
        # Extract assistant response
        if "ASSISTANT:" in response:
            response = response.split("ASSISTANT:")[-1].strip()
        
        return response
    
    def extract_text(self, image: Image.Image) -> str:
        """Extract all text from an image using OCR capability."""
        return self.analyze_image(
            image,
            "Extract all the text from this image. Preserve the formatting as much as possible."
        )
    
    def describe_image(self, image: Image.Image) -> str:
        """Get a detailed description of the image."""
        return self.analyze_image(
            image,
            "Describe this image in detail. Include information about text, diagrams, charts, and layout."
        )
    
    def extract_table(self, image: Image.Image) -> str:
        """Extract table data from an image."""
        return self.analyze_image(
            image,
            "Extract the table data from this image. Output in markdown table format."
        )

print("✅ VLM wrapper ready")
print("\nUsage:")
print("  vlm = VisionLanguageModel()")
print("  result = vlm.analyze_image(image, 'What is in this document?')")

---

## Part 4: Structured Information Extraction

In [ ]:
# Structured Extraction System

import json
import re

class StructuredExtractor:
    """
    Extract structured information from documents using VLM.
    """
    
    def __init__(self, vlm: VisionLanguageModel):
        self.vlm = vlm
    
    def extract(
        self,
        document: Document,
        schema: Dict[str, Any],
        schema_name: str = "extraction",
    ) -> Dict[str, Any]:
        """
        Extract structured data according to a schema.
        
        Args:
            document: Document to extract from
            schema: JSON schema defining fields to extract
            schema_name: Name of the schema (for prompting)
            
        Returns:
            Extracted data matching the schema
        """
        # Build extraction prompt
        schema_str = json.dumps(schema, indent=2)
        
        prompt = f"""Extract the following information from this document.

Schema ({schema_name}):
{schema_str}

Return a JSON object matching this schema exactly. Use null for missing values.
Only return the JSON, no other text."""
        
        # Process first page (or combine pages for multi-page docs)
        if len(document.pages) == 1:
            response = self.vlm.analyze_image(document.pages[0].image, prompt)
        else:
            # For multi-page, extract from each and merge
            all_extractions = []
            for page in document.pages[:5]:  # Limit to first 5 pages
                response = self.vlm.analyze_image(page.image, prompt)
                try:
                    extracted = self._parse_json(response)
                    all_extractions.append(extracted)
                except:
                    continue
            response = json.dumps(self._merge_extractions(all_extractions, schema))
        
        # Parse and validate response
        try:
            result = self._parse_json(response)
            return self._validate_against_schema(result, schema)
        except Exception as e:
            return {"error": str(e), "raw_response": response}
    
    def _parse_json(self, text: str) -> Dict:
        """Extract JSON from text that might contain other content."""
        # Try direct parse
        try:
            return json.loads(text)
        except:
            pass
        
        # Find JSON in text
        json_match = re.search(r'\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\}', text, re.DOTALL)
        if json_match:
            try:
                return json.loads(json_match.group())
            except:
                pass
        
        raise ValueError(f"Could not parse JSON from response")
    
    def _validate_against_schema(
        self,
        data: Dict,
        schema: Dict
    ) -> Dict:
        """Validate and coerce data to match schema."""
        result = {}
        
        for key, expected_type in schema.items():
            if key in data:
                result[key] = data[key]
            else:
                result[key] = None
        
        return result
    
    def _merge_extractions(
        self,
        extractions: List[Dict],
        schema: Dict
    ) -> Dict:
        """Merge extractions from multiple pages."""
        result = {}
        
        for key in schema:
            values = [e.get(key) for e in extractions if e.get(key) is not None]
            if values:
                # Take first non-null value (could be smarter)
                result[key] = values[0]
            else:
                result[key] = None
        
        return result

print("✅ Structured extraction system ready")

---

## Part 5: Multimodal RAG for Q&A

In [ ]:
# Multimodal RAG System

class MultimodalRAG:
    """
    RAG system that combines text and visual understanding.
    """
    
    def __init__(
        self,
        vlm: VisionLanguageModel,
        embedding_model: str = "BAAI/bge-m3",
    ):
        self.vlm = vlm
        self.embedding_model_name = embedding_model
        self._embedding_model = None
        self.documents: List[Document] = []
        self.page_descriptions: List[Dict] = []  # {doc_idx, page_num, description, embedding}
        self.index = None
    
    def add_document(self, document: Document):
        """Add a document to the knowledge base."""
        doc_idx = len(self.documents)
        self.documents.append(document)
        
        # Generate description for each page
        print(f"Processing {document.metadata['filename']}...")
        
        for page in document.pages:
            # Get visual description
            description = self.vlm.describe_image(page.image)
            
            # Combine with extracted text
            combined = f"{description}\n\nExtracted text:\n{page.text or 'No text extracted'}"
            
            self.page_descriptions.append({
                "doc_idx": doc_idx,
                "page_num": page.page_number,
                "description": combined,
            })
        
        # Rebuild index
        self._build_index()
        print(f"✅ Added {len(document.pages)} pages from {document.metadata['filename']}")
    
    def _build_index(self):
        """Build search index over page descriptions."""
        if not self.page_descriptions:
            return
        
        # Load embedding model
        if self._embedding_model is None:
            from sentence_transformers import SentenceTransformer
            self._embedding_model = SentenceTransformer(self.embedding_model_name, device="cuda")
        
        # Generate embeddings
        texts = [p["description"] for p in self.page_descriptions]
        embeddings = self._embedding_model.encode(texts)
        
        # Build FAISS index
        import faiss
        import numpy as np
        
        embeddings_array = np.array(embeddings).astype('float32')
        faiss.normalize_L2(embeddings_array)
        
        self.index = faiss.IndexFlatIP(embeddings_array.shape[1])
        self.index.add(embeddings_array)
    
    def answer_question(
        self,
        question: str,
        top_k: int = 3,
    ) -> Dict[str, Any]:
        """
        Answer a question using retrieved document context.
        
        Args:
            question: User's question
            top_k: Number of pages to retrieve
            
        Returns:
            Answer with sources and confidence
        """
        if self.index is None or not self.page_descriptions:
            return {"answer": "No documents loaded.", "sources": []}
        
        # Retrieve relevant pages
        query_embedding = self._embedding_model.encode([question])
        query_array = np.array(query_embedding).astype('float32')
        faiss.normalize_L2(query_array)
        
        scores, indices = self.index.search(query_array, top_k)
        
        # Get retrieved pages
        retrieved = []
        for idx, score in zip(indices[0], scores[0]):
            if idx >= 0:
                page_info = self.page_descriptions[idx]
                doc = self.documents[page_info["doc_idx"]]
                page = doc.pages[page_info["page_num"] - 1]
                retrieved.append({
                    "page": page,
                    "doc_name": doc.metadata["filename"],
                    "page_num": page_info["page_num"],
                    "score": float(score),
                    "description": page_info["description"],
                })
        
        # Use VLM to answer based on best matching page
        if retrieved:
            best_page = retrieved[0]["page"]
            
            prompt = f"""Based on this document page, answer the following question.
            
Question: {question}

Provide a detailed answer. If the information is not in the document, say so."""
            
            answer = self.vlm.analyze_image(best_page.image, prompt)
        else:
            answer = "No relevant information found in the documents."
        
        return {
            "answer": answer,
            "sources": [
                {
                    "document": r["doc_name"],
                    "page": r["page_num"],
                    "relevance": r["score"],
                }
                for r in retrieved
            ],
            "question": question,
        }

print("✅ Multimodal RAG system ready")

---

## Part 6: Complete Pipeline & API

In [ ]:
# Document Intelligence Pipeline

class DocumentIntelligencePipeline:
    """
    Complete pipeline for document intelligence.
    
    Combines:
    - Document ingestion
    - VLM analysis
    - Structured extraction
    - Q&A with RAG
    """
    
    def __init__(self, vlm_model: str = "llava-hf/llava-v1.6-34b-hf"):
        self.ingester = DocumentIngester()
        self.vlm = VisionLanguageModel(vlm_model)
        self.extractor = StructuredExtractor(self.vlm)
        self.rag = MultimodalRAG(self.vlm)
    
    def process_document(
        self,
        path: str,
        extract_schema: str = None,
        add_to_rag: bool = True,
    ) -> Dict[str, Any]:
        """
        Process a document through the full pipeline.
        
        Args:
            path: Path to document
            extract_schema: Schema name for extraction (or None)
            add_to_rag: Whether to add to RAG index
            
        Returns:
            Processing results
        """
        results = {"path": path, "status": "processing"}
        
        # 1. Ingest
        try:
            document = self.ingester.ingest(path)
            results["metadata"] = document.metadata
            results["num_pages"] = len(document.pages)
        except Exception as e:
            results["status"] = "error"
            results["error"] = f"Ingestion failed: {e}"
            return results
        
        # 2. Extract structured data
        if extract_schema and extract_schema in EXTRACTION_SCHEMAS:
            schema = EXTRACTION_SCHEMAS[extract_schema]
            results["extracted_data"] = self.extractor.extract(
                document, schema, extract_schema
            )
        
        # 3. Add to RAG
        if add_to_rag:
            self.rag.add_document(document)
            results["indexed"] = True
        
        results["status"] = "complete"
        return results
    
    def ask(self, question: str) -> Dict[str, Any]:
        """Ask a question about indexed documents."""
        return self.rag.answer_question(question)
    
    def extract(
        self,
        path: str,
        schema: Dict[str, Any]
    ) -> Dict[str, Any]:
        """Extract structured data from a document."""
        document = self.ingester.ingest(path)
        return self.extractor.extract(document, schema)

print("✅ Document Intelligence Pipeline ready")
print("\nUsage:")
print("  pipeline = DocumentIntelligencePipeline()")
print("  pipeline.process_document('invoice.pdf', extract_schema='invoice')")
print("  pipeline.ask('What is the total amount?')")

---

## 🎉 Checkpoint & Next Steps

You now have the foundation for a Multimodal Document Intelligence system:

- ✅ Document ingestion (PDF, images)
- ✅ Vision-Language Model integration
- ✅ Structured data extraction
- ✅ Multimodal RAG for Q&A
- ✅ Complete pipeline

### Implementation Roadmap

**Week 1-2:** Build document processing pipeline
- [ ] Implement ingestion for all formats
- [ ] Test VLM on various document types
- [ ] Create extraction schemas for your domain

**Week 3:** Build RAG and Q&A
- [ ] Implement multimodal RAG
- [ ] Add table extraction
- [ ] Create API endpoints

**Week 4:** Optimization
- [ ] Optimize for batch processing
- [ ] Add caching
- [ ] Profile memory usage

**Week 5-6:** Evaluation & Documentation
- [ ] Create benchmark dataset
- [ ] Measure accuracy
- [ ] Write documentation

---

In [ ]:
# Cleanup
import gc

if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

print("✅ Ready to build your Document Intelligence system!")